In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
data = pd.read_excel('../dataset.xls')
data.drop('Unnamed: 0', axis = 1, inplace = True)
close_value_mean =data['Close_Value'].mean()
data['Close_Value'].fillna(close_value_mean, inplace = True)
data.head()

,Customer,Agent,SalesAgentEmailID,ContactEmailID,Stage,Product,Close_Value,Created Date,Close Date
0,Konex,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255.000000,2016-01-04,2016-05-24
1,Finjob,Rosalina Dieter,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45.000000,2016-01-04,2016-11-02
2,Kinnamplus,Donn Cantrell,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1852.915504,2016-01-06,2016-12-07
3,Genco Pura Olive Oil Company,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054.000000,2016-01-06,2016-09-11
4,Stanredtax,Lajuana Vencill,lajuana@piedpiper.com,candice@stanredtax.com,Won,MG Advanced,3180.000000,2016-01-07,2016-01-31


In [3]:
data['Stage'] = data['Stage'].astype('category')
data['Customer'] = data['Customer'].astype('category')
data['Agent'] = data['Agent'].astype('category')
data['SalesAgentEmailID'] = data['SalesAgentEmailID'].astype('category')
data['ContactEmailID'] = data['ContactEmailID'].astype('category')
data['Product'] = data['Product'].astype('category')

data.dtypes



Customer                   category
Agent                      category
SalesAgentEmailID          category
ContactEmailID             category
Stage                      category
Product                    category
Close_Value                 float64
Created Date         datetime64[ns]
Close Date           datetime64[ns]
dtype: object

In [4]:
data['Stage']=data['Stage'].cat.codes
data['Customer']=data['Customer'].cat.codes
data['Agent']=data['Agent'].cat.codes
data['SalesAgentEmailID']=data['SalesAgentEmailID'].cat.codes
data['ContactEmailID']=data['ContactEmailID'].cat.codes
data['Product']=data['Product'].cat.codes

In [6]:
data['Created Date'] = data['Created Date'].astype(int)
data['Close Date'] = data['Close Date'].astype(int)

In [7]:
# droped_data = data.drop(['Stage', 'Created Date' , 'Close Date'],axis=1)
# data

In [7]:
X = data.drop('Stage' , axis=1)
y = data['Stage']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1) # 85% training and 15% test


In [8]:
model = tree.DecisionTreeClassifier() 
classifired_data = model.fit(X_train, y_train)

In [24]:
# tree.plot_tree(classifired_data)

In [9]:
y_pred = classifired_data.predict(X_test)

In [10]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("F1:",metrics.f1_score(y_test, y_pred,average='macro'))

Accuracy: 0.6760683760683761
F1: 0.6204881267431744


In [12]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.49      0.55      0.52       300
           1       0.48      0.45      0.46       309
           2       0.89      0.87      0.88       561

    accuracy                           0.68      1170
   macro avg       0.62      0.62      0.62      1170
weighted avg       0.68      0.68      0.68      1170

[[164 113  23]
 [135 138  36]
 [ 37  35 489]]
